In [140]:
import pandas as pd
#from caserec.recommenders.hybrid import MMR
from sklearn.cluster import KMeans
from caserec.recommenders.item_recommendation.userknn import UserKNN
from caserec.recommenders.item_recommendation.itemknn import ItemKNN
import urllib.parse
# Load your dataset
from sqlalchemy import create_engine
from sqlalchemy import text

In [175]:
# Load the dataset
params = urllib.parse.quote_plus(r'Driver={ODBC Driver 18 for SQL Server};Server=tcp:adaptive-learning-server.database.windows.net,1433;Database=adaptive_learning_db;Uid=superadmin;Pwd=Poorpassword@2024;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine_azure = create_engine(conn_str,echo=True)

connection = engine_azure.connect()
print("Connection successful !!!!")

students_query = text(
    "SELECT id, age, gender, study_field, education_level, employment, study_place "
    "FROM import_student;"
)

students_result = connection.execute(students_query)

# Fetch all the rows and convert them to a Pandas DataFrame
students_df = pd.DataFrame(students_result.fetchall(), columns=students_result.keys())
print("students_df : !!!!! ", students_df)

2024-01-31 14:00:07,052 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2024-01-31 14:00:07,053 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-31 14:00:07,075 INFO sqlalchemy.engine.Engine SELECT schema_name()
2024-01-31 14:00:07,075 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ()
2024-01-31 14:00:07,144 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2024-01-31 14:00:07,144 INFO sqlalchemy.engine.Engine [generated in 0.00056s] ()
2024-01-31 14:00:07,164 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2024-01-31 14:00:07,164 INFO sqlalchemy.engine.Engine [generated in 0.00052s] ()
Connection successful !!!!
2024-01-31 14:00:07,204 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-31 14:00:07,205 INFO sqlalchemy.engine.Engine SELECT id, age, gender, study_field, education_level, employment, study_place FROM import_stud

In [142]:
ratings_query = text(
    "SELECT user_id, item_id, rating "
    "FROM rating_materials;"
)

ratings_result = connection.execute(ratings_query)

# Fetch all the rows and convert them to a Pandas DataFrame
ratings_df = pd.DataFrame(ratings_result.fetchall(), columns=ratings_result.keys())
print("ratings_df : !!!!! ", ratings_df)

2024-01-31 13:13:21,778 INFO sqlalchemy.engine.Engine SELECT user_id, item_id, rating FROM rating_materials;
2024-01-31 13:13:21,779 INFO sqlalchemy.engine.Engine [generated in 0.00117s] ()
ratings_df : !!!!!       user_id  item_id  rating
0        147        1       1
1        148        1       1
2        149        1       1
3        150        1       6
4        151        1       1
..       ...      ...     ...
433      215        6       6
434      216        6       4
435      217        6       6
436      218        6       5
437      219        6       2

[438 rows x 3 columns]


In [143]:
merged_df = pd.merge(ratings_df, students_df, how='inner', left_on='user_id', right_on='id')
print("merged_df : !!!!! ", merged_df)

merged_df : !!!!!       user_id  item_id  rating   id  age  gender  study_field  education_level  \
0        147        1       1  147   20       1            1                1   
1        147        2       2  147   20       1            1                1   
2        147        3       3  147   20       1            1                1   
3        147        4       4  147   20       1            1                1   
4        147        5       5  147   20       1            1                1   
..       ...      ...     ...  ...  ...     ...          ...              ...   
433      219        2       5  219   16       1            4                1   
434      219        3       4  219   16       1            4                1   
435      219        4       1  219   16       1            4                1   
436      219        5       3  219   16       1            4                1   
437      219        6       2  219   16       1            4                1   

     emp

In [144]:
features_for_clustering = ['id','age', 'gender', 'study_field', 'education_level', 'employment', 'study_place']
user_item_matrix = merged_df.pivot_table(index='user_id', columns='item_id', values='rating', fill_value=0)


In [156]:
# Step 2: Apply Clustering Algorithm (e.g., K-means) on Demographic Data
# print("students_df : !!!!! ", students_df)
demographic_data = students_df[features_for_clustering].drop_duplicates().set_index('id')
num_clusters = 9  # Adjust the number of clusters as needed
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
user_clusters = kmeans.fit_predict(demographic_data)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [178]:
new_user_demographics = {
    'age': 25,
    'gender': '1',
    'study_field': '1',
    'education_level': '1',
    'employment': '1',
    'study_place': '1'
}

In [158]:
num_clusters = 9
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
user_clusters = kmeans.fit_predict(demographic_data)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [159]:
# Assign the new user to a demographic cluster
new_user_cluster = kmeans.predict([list(new_user_demographics.values())])[0]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


In [160]:
cluster_members = demographic_data.index[user_clusters == new_user_cluster]

# Prepare Input for CaseRecommender

In [161]:
train_file = 'train_file.dat'  # File to store training data in CaseRecommender format
with open(train_file, 'w') as file:
    for user in cluster_members:
        items_rated = merged_df[merged_df['id'] == user][['item_id', 'rating']].values
        for item, rating in items_rated:
            file.write(f"{user}::{item}::{rating}\n")

In [150]:
train = ItemKNN(train_file, similarity_metric='cosine', sep='::')
train.compute()

[Case Recommender: Item Recommendation > ItemKNN Algorithm]

train data:: 6 users and 6 items (36 interactions) | sparsity:: 0.00%
training_time:: 0.001144 sec
prediction_time:: 0.000108 sec




In [177]:
# Read the train_file and store the data in a dictionary
train_file_path = 'train_file.dat'

similar_users_ratings = {}
with open(train_file_path, 'r') as file:
    for line in file:
        user_id, item_id, rating = map(int, line.strip().split('::'))
        if item_id not in similar_users_ratings:
            similar_users_ratings[item_id] = []
        similar_users_ratings[item_id].append(rating)

# Calculate the absolute difference of each item's average rating from 1
rating_diff_from_one = {}
for item_id, ratings in similar_users_ratings.items():
    average_rating = sum(ratings) / len(ratings)
    diff_from_one = abs(average_rating - 1)
    rating_diff_from_one[item_id] = diff_from_one

# Find the item with the minimum absolute difference from 1

best_item_id = min(rating_diff_from_one, key=rating_diff_from_one.get)
best_rating = rating_diff_from_one[best_item_id]
finalrec_query = text(
    "INSERT INTO recommendation_table (id, item_id) "
    "VALUES (1, :best_item_id);;"
)

# Print or use the best item and its adjusted average rating
print(f"Best Recommended Item: {best_item_id}")


Best Recommended Item: 1
